In [23]:
import pandas as pd
import glob
from IPython.display import display

In [35]:
# getting OccurenceID
def merge_data(csv_file_path):
    global df_all
    global df_meta_all
    df = pd.read_csv(csv_file_path, encoding='utf-8')
    df['occurrenceID'] = df['collectionCode']+ '_' + df['catalogNumber'].apply(str)
    df_meta = df[['occurrenceID','publicationTitle', 'publicationYear', 'collectionCode', 
                 'catalogNumber','publicationAuthors', 'authorityURI', 'authorityValue']].drop_duplicates()
    df.drop(columns=['publicationTitle', 'publicationYear', 'collectionCode', 
                 'catalogNumber','publicationAuthors', 'authorityURI', 'authorityValue'], inplace=True )
    return (df, df_meta)
# Getting Metadata

directory_path  = "./data/final_final_data/**/"  # Replace with the path to your folder
all_csv_files_path  = glob.glob(f"{directory_path}/*.csv", recursive=True)
# Display the list of CSV files
print(f"CSV files in the folder:{len(all_csv_files_path)}")

df_all = pd.DataFrame()
df_meta_all = pd.DataFrame()
for csv_file_path in all_csv_files_path: #sorted(all_csv_files_path, key=lambda x: Path(x).stem):
    print(csv_file_path)
    df, df_meta = merge_data(csv_file_path)
    df_all = pd.concat([df_all, df])
    df_meta_all = pd.concat([df_meta_all, df_meta])

df_all.drop_duplicates(inplace=True)
df_meta_all.drop_duplicates(inplace=True)
df_all.to_csv('./gbif_test_data/test.csv', index=False)
df_meta_all.to_csv('./gbif_test_data/test_meta.csv', index=False)

In [87]:
import pandas as pd
import hashlib

# Read the large CSV file
df = pd.read_csv('./gbif_test_data/test.csv')
df_event_all = pd.DataFrame()


# Group by the unique ID column and save each group to a separate CSV file
for unique_id, group in df.groupby('occurrenceID'):
    df_event_occur = pd.DataFrame()
    group_reset = group.reset_index(drop=True)
    group.drop(columns=['eventDate'], inplace=True)
    # print(group_reset.index.tolist())
    group['occurrenceID'] = group['occurrenceID']+'_'+ group_reset.index.astype(str)
    for unique_id_event, group_event in group.groupby(by=['verbatimLocality', 'year']):
        group_event_occr = group_event.drop(columns=['decimalLatitude', 'decimalLongitude', 'year'])
        group_event = group_event[['verbatimLocality', 'decimalLatitude', 'decimalLongitude', 'year']]
        # event_id for events
        event_id = unique_id + '_' + str(int(hashlib.sha256(unique_id_event[0].encode('utf-8')).hexdigest(), 16) % 10**8)
        group_event.loc[:,['eventID']] =  event_id
        group_event_occr = group_event_occr.join(other=group_event, how='left', lsuffix='_left')
        df_event_all = pd.concat([df_event_all, group_event])
        group_event_occr.drop(columns=['verbatimLocality', 'verbatimLocality_left'], inplace=True)
        # group_event_occr.loc[:, ['eventID']] = 
        group_event_occr.to_csv(f'./gbif_test_data/eventdata/Event_{event_id}.csv', index=False)
df_event_all.drop_duplicates(inplace=True)
df_event_all.to_csv('./gbif_test_data/'+'event_all.csv', index=False)

In [2]:
# Merge event data files based on tuexen number 

import pandas as pd
import os


directory_path = './gbif_test_data/eventdata/'
filename_list = [filename for filename in os.listdir(directory_path)]
tuexen_unique_no_list = set([filename.split('_')[2] for filename in os.listdir(directory_path) if filename.endswith('.csv')])
print(tuexen_unique_no_list)
# print(filename_list, tuexen_unique)

for tuexen_unique_no in tuexen_unique_no_list:
    dfs = []
    for filename in os.listdir(directory_path):
        try:
            if tuexen_unique_no in filename:
                df = pd.read_csv(os.path.join(directory_path, filename))
                dfs.append(df)
        except Exception as e:
            print(tuexen_unique_no)
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df.to_csv('./gbif_test_data/eventdata_combined/'+ 'Event_'+ tuexen_unique_no+ '.csv', index=False)

{'8257', '8568', '9506', '2673', '8156', '3156', '7465', '3006', '7870', '3146', '3152', '2981', '2983', '7812', '11868', '2929', '2982', '3131', '7487'}


In [4]:
import pandas as pd 

def remove_linefeed(text):
    text = text.replace('\n', ' ')
    return text

df = pd.read_csv('./gbif_test_data/event_all.csv', encoding='utf-8')
df['verbatimLocality'] = df['verbatimLocality'].apply(remove_linefeed)
df.to_csv('./gbif_test_data/'+'event_all_removed_linefeed.csv', index=False)

In [2]:
# Parse and complete xml file
import xml.etree.ElementTree as ET
import pandas as pd

df = pd.read_csv('./gbif_test_data/test_meta.csv', encoding='utf-8')

for _ , row in df.iterrows():
    tree =  ET.parse('./gbif_test_data/eml-test-v1.0.xml')
    root = tree.getroot()
    # for title in root.findall('title'):
    root.find('dataset').find('title').text =  row['publicationTitle']
    print(root.find('dataset').find('title').text)
    tree.write('./gbif_test_data/meta_xml/meta_eml'+row['occurrenceID']+'.xml', encoding='utf-8', xml_declaration=True)

Najas minor all. in Europa einst und jetzt
Über einige bemerkenswerte Pflanzenvereine in Litauen
Waldgesellschaften des nördlichen Havellandes
Die Algenflora Westfalens und der angrenzenden Gebiete
Das Klimax-Gebiet der Buchenwälder in den Westkarpathen
Die Pflanzengesellschaften des westsächsischen Berg- und Hügellandes (Flußgebiet der Freiberger und Zwickauer Mulde) 2. Teil Die Pflanzengesellschaften der erzgebirgischen Moore
Ein Beitrag zur geobotanischen Durchforschung des Steppengebiets im Böhmischen Mittelgebirge
Die Pflanzengesellschaften des westsächsischen Berg- und Hügellandes (Flußgebiet der Freiberger und Zwickauer Mulde) 3. Teil Laichkraut-, Röhricht- und Großseggengesellschaften, 4. Teil Die Pflanzengesellschaften der Quellfluren und Bachufer und der Verband der Schwarzerlen-Gesellschaften
Zur Kenntnis der Waldgesellschaften im Böhmischen Mittelgebirge (Wälder des Milleschauer Mittelgebirges)
Pflanzensoziologische Untersuchungen in der Alpinen Stufe Finnisch-Lapplands
Der